In [1]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

In [2]:
def gaussian_blur_canny(img):
    # 高斯滤波核大小
    blur_ksize = 5
    # Canny边缘检测高低阈值
    canny_lth = 50
    canny_hth = 150
    # 1. 灰度化、滤波和Canny
    gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    blur_gray = cv2.GaussianBlur(gray, (blur_ksize, blur_ksize), 1)
    return cv2.Canny(blur_gray, canny_lth, canny_hth)

In [3]:
def roi_mask(img):
    # 创建掩膜
    mask = np.zeros_like(img)
    corner_points = np.array([[
        [0, mask.shape[0]],
        [mask.shape[1], mask.shape[0]],
        [520, 325],
        [460, 325]
    ]])
    cv2.fillPoly(mask, corner_points, 255)
    masked_img = cv2.bitwise_and(img, mask)
    return masked_img

In [4]:
def left_right_lane(img):
    # 统计概率霍夫直线变换
    rho = 1
    theta = np.pi / 180
    threshold = 15
    min_line_len = 40
    max_line_gap = 20
    lines = cv2.HoughLinesP(img, rho, theta, threshold, minLineLength=min_line_len, maxLineGap=max_line_gap)
    # 划分左右车道
    left_lines, right_lines = [], []
    for line in lines:
        x1, y1, x2, y2 = line[0]
        if (y2 - y1)/(x2 - x1) < 0:
            left_lines.append(line[0])
        else:
            right_lines.append(line[0])
    return left_lines, right_lines

In [5]:
def clean_lines(lines, threshold):
    # 迭代计算斜率均值，排除掉与差值差异较大的数据
    slope = [(y2 - y1) / (x2 - x1) for x1, y1, x2, y2 in lines]
    while len(lines) > 0:
        mean = np.mean(slope)
        diff = [abs(s - mean) for s in slope]
        idx = np.argmax(diff)
        if diff[idx] > threshold:
            slope.pop(idx)
            lines.pop(idx)
        else:
            break

In [6]:
def get_points(lines):
    point = [[x1, y1] for x1, y1, x2, y2 in lines]
    point += [[x2, y2] for x1, y1, x2, y2 in lines]
    return np.array(point)

In [7]:
def least_squares_fit(img, points):
    fit = np.polyfit(points[:,1], points[:,0], 1)
    fit_fn = np.poly1d(fit)
    return [[fit_fn(325), 325],[fit_fn(img.shape[0]), img.shape[0]]]

In [8]:
def get_lane(img, left_results, right_results):
    vtxs = np.array([[left_results[1], left_results[0], right_results[0], right_results[1]]], dtype=np.int)
    img_copy = np.zeros_like(img)
    return cv2.fillPoly(img_copy, vtxs, (0, 255, 0))

In [9]:
def get_lane_img(img):
    img1 = gaussian_blur_canny(img)
    img1 = roi_mask(img1)
    left_lines, right_lines = left_right_lane(img1)
    clean_lines(left_lines, 0.1)
    clean_lines(right_lines, 0.1)
    left_point, right_point = get_points(left_lines), get_points(right_lines)
    if len(left_point) == 0 or len(right_point) == 0:
        return img
    left_results, right_results = least_squares_fit(img, left_point), least_squares_fit(img, right_point)
    img_lane = get_lane(img, left_results, right_results)
    return cv2.addWeighted(img, 1, img_lane, 0.2, 0)

In [10]:
from moviepy.editor import VideoFileClip
# 运行下面的代码需要 moviepy
# pip install moviepy
output = 'output.mp4'
clip = VideoFileClip("video.mp4")
out_clip = clip.fl_image(get_lane_img)
out_clip.write_videofile(output, audio=False)

t:   1%|          | 5/681 [00:00<00:14, 47.03it/s, now=None]

Moviepy - Building video output.mp4.
Moviepy - Writing video output.mp4



Moviepy - Done !
Moviepy - video ready output.mp4
